In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import mlflow
import optuna
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [2]:
import mlflow
#Set up the MLflow tracking server
mlflow.set_tracking_uri('http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/')

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp5: ML Algos with Hyperparameter tuning")

<Experiment: artifact_location='s3://youtubecommentmlflow/149143573475036011', creation_time=1741969037519, experiment_id='149143573475036011', last_update_time=1741969037519, lifecycle_stage='active', name='Exp5: ML Algos with Hyperparameter tuning', tags={}>

In [ ]:
df = pd.read_csv('C:/test/youtube_comment_sentiment_analysis/Artifacts/youtubecomment_preprocessing.csv')
df.shape

(15833, 2)

In [ ]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['Sentiment'] = df['Sentiment'].map({-1: 2, 0: 0, 1: 1})

ngram_range = (1, 3)  # Trigram setting
max_features = 8000  # Set max_features to 8000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['Comment'], df['Sentiment'], test_size=0.2, random_state=42, stratify=df['Sentiment'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=15)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2025-03-14 22:30:37,169] A new study created in memory with name: no-name-d51fecf4-9c78-45f6-9d19-dd59c9d28b06
[I 2025-03-14 22:32:00,186] Trial 0 finished with value: 0.4704767919166404 and parameters: {'n_estimators': 259, 'learning_rate': 0.00015557248069403339, 'max_depth': 4}. Best is trial 0 with value: 0.4704767919166404.
[I 2025-03-14 22:33:24,338] Trial 1 finished with value: 0.5648879065361541 and parameters: {'n_estimators': 101, 'learning_rate': 0.0012132948481815404, 'max_depth': 10}. Best is trial 1 with value: 0.5648879065361541.
[I 2025-03-14 22:36:06,459] Trial 2 finished with value: 0.6510893590148406 and parameters: {'n_estimators': 285, 'learning_rate': 0.02238137488721406, 'max_depth': 7}. Best is trial 2 with value: 0.6510893590148406.
[I 2025-03-14 22:38:00,255] Trial 3 finished with value: 0.5131038838017051 and parameters: {'n_estimators': 196, 'learning_rate': 0.0017840097133948432, 'max_depth': 6}. Best is trial 2 with value: 0.6510893590148406.
[I 2025-03

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011/runs/a026859d8986463f92fc9e426548e122
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011
